# Deep Hybrid Model: VGG16 + GMM
Inspiration: https://towardsdatascience.com/a-simple-way-to-detect-anomaly-3d5a48c0dae0

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.mixture import GaussianMixture
from tensorflow.keras.datasets import mnist
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import cv2
import numpy as np

import time

In [2]:
# Initialize VGG16 with weights from training on ImageNet
# Input channels must be: 3
# Input size must be greater than 32 -> thus 2*28 -> 56
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(56, 56, 3))
conv_out = vgg_conv.get_layer('block2_conv2').output
avgpool = layers.GlobalAveragePooling2D()(conv_out)
model = models.Model(inputs=vgg_conv.input, outputs=avgpool)

All instances

In [4]:
gmm = GaussianMixture(n_components=1)
start = time.process_time()

features = model.predict(x_normal)
gmm.fit(features)

end = time.process_time()
print(end - start)

start = time.process_time()

test_features = model.predict(x_contam)
score = gmm.score_samples(test_features)

end = time.process_time()
print(end - start)

116.921875
204.03125


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Map 28x28x1 into 56x56x3
# Extrapolate and copy image into all three channels
def reshape_x(x):
    new_x = np.empty((len(x), 56, 56))
    for i, e in enumerate(x):
        new_x[i] = cv2.resize(e, (56, 56))

    new_x = np.expand_dims(new_x, axis=-1)
    new_x = np.repeat(new_x, 3, axis=-1)
    return new_x


REPETITIONS = 10

evals = np.zeros((10, REPETITIONS))
for i in range(10):
    for j in tqdm_notebook(range(REPETITIONS)):
        normal = i

        x_normal = x_train[y_train == normal]

        x_normal = reshape_x(x_normal)
        x_contam = reshape_x(x_test)


        # One normal class
        gmm = GaussianMixture(n_components=1)
        # Fit GMM to features from the normal class
        features = model.predict(x_normal)
        gmm.fit(features)

        # Compute features from contaminated test case
        test_features = model.predict(x_contam)
        # Compute scores from contaminated features
        score = gmm.score_samples(test_features)

        # Normalize the scores between 0 and 1
        score = ((score * -1 - np.abs(np.max(score)) ))/(np.abs(np.min(score)))

        # Create labels
        labels = np.copy( y_test )
        labels[y_test == normal] = 0
        labels[y_test != normal] = 1

        # Compute AUC-ROC
        AUC = roc_auc_score(labels, score)
        evals[i,j] = AUC
    print(np.mean(evals[i,:])*100)
        
print(np.mean(evals, axis=1))
print(np.std(evals, axis=1))

KeyboardInterrupt: 

In [9]:
print(np.mean(evals, axis=1)*100)
print(np.std(evals, axis=1)*100)

[96.20028056 99.84886364 93.31845917 97.67303605 94.1653579  94.98334387
 96.00957536 94.9202751  92.79636378 94.81276233]
[1.11022302e-14 2.22044605e-14 0.00000000e+00 2.22044605e-14
 2.22044605e-14 1.11022302e-14 1.11022302e-14 1.11022302e-14
 1.11022302e-14 0.00000000e+00]


Percentage-contaminated test data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def create_dataset_VGG16(x_test, y_test, normal=4, contamination=0.10):
    

    # random contam indices
    contamination = 0.10
    num_normals   = int( np.sum(y_test == normal) )
    x_temp = x_train[y_train != normal]
    num_outliers = int(np.ceil( num_normals * contamination ))
    idx = np.random.randint(low=0, high=len(x_temp), size=num_outliers)

    x_contam = np.zeros((num_normals + num_outliers, 28, 28))
    x_contam[:num_normals] = x_test[ y_test == normal ]
    x_contam[num_normals:] = x_temp[idx]

    y_contam = np.zeros((num_normals + num_outliers))
    y_contam[:num_normals] = normal
    y_contam[num_normals:] = y_train[y_train != normal][idx]
    
    return (x_contam, y_contam)


# Map 28x28x1 into 56x56x3
# Extrapolate and copy image into all three channels
def reshape_x(x):
    new_x = np.empty((len(x), 56, 56))
    for i, e in enumerate(x):
        new_x[i] = cv2.resize(e, (56, 56))

    new_x = np.expand_dims(new_x, axis=-1)
    new_x = np.repeat(new_x, 3, axis=-1)
    return new_x


REPETITIONS = 10

evals = np.zeros((10, REPETITIONS))
for i in range(10):
    for j in tqdm_notebook(range(REPETITIONS)):
        normal = i
        
        (x_contam, y_contam) = create_dataset_VGG16(x_test, y_test, normal=normal)

        x_normal = x_train[y_train == normal]

        x_normal = reshape_x(x_normal)
        x_contam = reshape_x(x_contam)


        # One normal class
        gmm = GaussianMixture(n_components=1)
        # Fit GMM to features from the normal class
        features = model.predict(x_normal)
        gmm.fit(features)

        # Compute features from contaminated test case
        test_features = model.predict(x_contam)
        # Compute scores from contaminated features
        score = gmm.score_samples(test_features)

        # Normalize the scores between 0 and 1
        score = ((score * -1 - np.abs(np.max(score)) ))/(np.abs(np.min(score)))

        # Create labels
        labels = np.copy( y_contam )
        labels[y_contam == normal] = 0
        labels[y_contam != normal] = 1

        # Compute AUC-ROC
        AUC = roc_auc_score(labels, score)
        evals[i,j] = AUC
    print(evals[i,:])
        
print(np.mean(evals, axis=1))
print(np.std(evals, axis=1))


[0.96712828 0.95484173 0.9645252  0.95775718 0.9608913  0.96202624
 0.9574344  0.96974177 0.95856935 0.96121408]



[0.99876343 0.99870933 0.99879434 0.99883299 0.99898756 0.99874024
 0.99860886 0.99861659 0.99870933 0.99896437]



[0.92702743 0.93689438 0.92888156 0.92875112 0.941553   0.92395274
 0.93492844 0.94387299 0.94827072 0.93258982]



[0.9778061  0.96410156 0.97536516 0.97743358 0.97693363 0.97925694
 0.9711989  0.9781394  0.97853152 0.97380649]



[0.93935279 0.9318439  0.95296139 0.95881421 0.93687383 0.93770701
 0.94170833 0.93714127 0.94027855 0.94681026]



[0.94160438 0.95287743 0.95815894 0.94865471 0.93163926 0.95269058
 0.94367215 0.9454285  0.9607997  0.95107125]



[0.94217554 0.96472686 0.96465075 0.96652096 0.95682194 0.96860865
 0.97286013 0.95319024 0.95852905 0.95527792]



[0.95530958 0.95123909 0.96754939 0.94531752 0.96658608 0.94939745
 0.94310755 0.95418571 0.94071814 0.94161535]



[0.92477895 0.93308679 0.93020576 0.92523991 0.93387252 0.93046767
 0.93822026 0.92048359 0.92509324 0.93911076]



[0.94547096 0.94466632 0.95530326 0.92558067 0.94564759 0.94444063
 0.94042724 0.94682511 0.95405705 0.9389259 ]
[0.96141295 0.9987727  0.93467222 0.97525733 0.94234915 0.94865969
 0.9603362  0.95150259 0.93005594 0.94413447]
[0.00439168 0.00012141 0.0075379  0.00438297 0.00778657 0.00807624
 0.0085059  0.00913235 0.00583    0.00787406]


In [4]:
print(np.mean(evals, axis=1) * 100)
print(np.std(evals, axis=1) * 100)

[96.14129529 99.87727027 93.46722197 97.52573277 94.23491534 94.86596911
 96.03362039 95.15025877 93.00559443 94.41344729]
[0.43916812 0.0121405  0.75379022 0.43829715 0.77865682 0.80762442
 0.85058976 0.9132351  0.58300007 0.78740569]
